In [1]:
import torch
import sys
import yaml
from torchvision import transforms, datasets
import torchvision
import numpy as np
import os
from sklearn import preprocessing
from torch.utils.data.dataloader import DataLoader

In [2]:
import tqdm

In [3]:
with tqdm.notebook.tqdm(total=100) as pbar:     
     for i in range(10):
         pbar.update(10)
         pbar.refresh()

  0%|          | 0/100 [00:00<?, ?it/s]

In [4]:
# From Yao load_and_convert
import torch
from torchvision import models

class ResNet(torch.nn.Module):
    def __init__(self, net_name, pretrained=False, use_fc=False):
        super().__init__()
        base_model = models.__dict__[net_name](pretrained=pretrained)
        self.encoder = torch.nn.Sequential(*list(base_model.children())[:-1])

        self.use_fc = use_fc
        if self.use_fc:
            self.fc = torch.nn.Linear(2048, 512)

    def forward(self, x):
        x = self.encoder(x)
        x = torch.flatten(x, 1)
        if self.use_fc:
            x = self.fc(x)
        return x
    
device = torch.device('cuda:1')
model = ResNet('resnet50', pretrained=False, use_fc=False).to(device)
model_path = '../ckpt/detconb/04_20_23-31/04_20_23-31_resnet50_300.pth.tar'
pth_file = torch.load(model_path, map_location=device)
checkpoint = torch.load(model_path, map_location=device)['model']#['online_backbone']
state_dict = {}
length = len(model.encoder.state_dict())
for name, param in zip(model.encoder.state_dict(), list(checkpoint.values())[:length]):
    state_dict[name] = param
model.encoder.load_state_dict(state_dict, strict=True)
#model =  torch.nn.DataParallel(model, device_ids=[1, 2, 3,4,5,6,7,8])
model.eval()

ResNet(
  (encoder): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (0): Conv2d(64, 2

In [5]:
batch_size = 512 
data_transforms = torchvision.transforms.Compose([
                                                    transforms.Resize((224, 224)), #FIXME: They only did smallest side resize to 224
                                                    transforms.ToTensor(),
                                                    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
                                                    
                                                ])

In [6]:
# config = yaml.load(open("../config/config.yaml", "r"), Loader=yaml.FullLoader)

In [7]:
train_dataset = datasets.ImageFolder('../imagenet/images/train/', transform=data_transforms)

test_dataset = datasets.ImageFolder('../imagenet/images/val/', transform=data_transforms)

In [8]:
print("Input shape:", train_dataset.__getitem__(0)[0].shape)
print("Input shape:", test_dataset.__getitem__(0)[0].shape)

Input shape: torch.Size([3, 224, 224])
Input shape: torch.Size([3, 224, 224])


In [9]:
train_loader = DataLoader(train_dataset, batch_size=batch_size,
                          num_workers=8, drop_last=False, shuffle=True)

test_loader = DataLoader(test_dataset, batch_size=batch_size,
                          num_workers=8, drop_last=False, shuffle=True)

In [10]:
# device = 'cpu' #'cuda' if torch.cuda.is_available() else 'cpu'
# encoder = ResNet18(**config['network'])
encoder = model
output_feature_dim = 2048 #encoder.projetion.net[0].in_features

In [11]:
class LogisticRegression(torch.nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegression, self).__init__()
        self.linear = torch.nn.Linear(input_dim, output_dim)
        
    def forward(self, x):
        return self.linear(x)

In [12]:
logreg = LogisticRegression(output_feature_dim, 1000)
logreg = logreg.to(device)

In [13]:

def get_features_from_encoder(encoder, loader):
    
    x_train = []
    y_train = []

    # get the features from the pre-trained model
    for (x, y) in tqdm.notebook.tqdm((loader)):
        with torch.no_grad():
            feature_vector = encoder(x.to(device))
            x_train.extend(feature_vector.cpu())
            y_train.extend(y.cpu().numpy())

    x_train = torch.stack(x_train)
    y_train = torch.tensor(y_train)
    return x_train, y_train

In [14]:
encoder.eval()
print("Getting Train Features")
x_train, y_train = get_features_from_encoder(encoder, train_loader)
# print("Getting Test Features")
# x_test, y_test = get_features_from_encoder(encoder, test_loader)


Getting Train Features


  0%|          | 0/2503 [00:00<?, ?it/s]

In [15]:
print("Getting Test Features")
x_test, y_test = get_features_from_encoder(encoder, test_loader)

Getting Test Features


  0%|          | 0/98 [00:00<?, ?it/s]

In [16]:
if len(x_train.shape) > 2:
    x_train = torch.mean(x_train, dim=[2, 3])
    x_test = torch.mean(x_test, dim=[2, 3])
    
print("Training data shape:", x_train.shape, y_train.shape)
print("Testing data shape:", x_test.shape, y_test.shape)

Training data shape: torch.Size([1281167, 2048]) torch.Size([1281167])
Testing data shape: torch.Size([50000, 2048]) torch.Size([50000])


In [17]:
if len(x_train.shape) > 2:
    x_train = torch.mean(x_train, dim=[2, 3])
    x_test = torch.mean(x_test, dim=[2, 3])
    
print("Training data shape:", x_train.shape, y_train.shape)
print("Testing data shape:", x_test.shape, y_test.shape)


Training data shape: torch.Size([1281167, 2048]) torch.Size([1281167])
Testing data shape: torch.Size([50000, 2048]) torch.Size([50000])


In [18]:
#pip install --upgrade ipywidgets

In [19]:
def create_data_loaders_from_arrays(X_train, y_train, X_test, y_test):

    train = torch.utils.data.TensorDataset(X_train, y_train)
    train_loader = torch.utils.data.DataLoader(train, batch_size=100, shuffle=True)

    test = torch.utils.data.TensorDataset(X_test, y_test)
    test_loader = torch.utils.data.DataLoader(test, batch_size=512, shuffle=False)
    return train_loader, test_loader

In [20]:
# They didn't do this!!!
# scaler = preprocessing.StandardScaler()
# scaler.fit(x_train)
# x_train = scaler.transform(x_train).astype(np.float32)
# x_test = scaler.transform(x_test).astype(np.float32)

In [21]:
train_loader, test_loader = create_data_loaders_from_arrays(x_train, y_train, x_test, y_test)
#train_loader, test_loader = create_data_loaders_from_arrays(torch.from_numpy(x_train), y_train, torch.from_numpy(x_test), y_test)

In [22]:
optimizer = torch.optim.Adam(logreg.parameters(), lr=3e-4)
criterion = torch.nn.CrossEntropyLoss()
eval_every_n_epochs = 10

for epoch in tqdm.notebook.tqdm((range(100))):
#     train_acc = []
    for x, y in train_loader:

        x = x.to(device)
        y = y.to(device)
        
        # zero the parameter gradients
        optimizer.zero_grad()        
        
        logits = logreg(x)
        predictions = torch.argmax(logits, dim=1)
        
        loss = criterion(logits, y)
        
        loss.backward()
        optimizer.step()
    
    total = 0
    if epoch % eval_every_n_epochs == 0:
        correct = 0
        for x, y in test_loader:
            x = x.to(device)
            y = y.to(device)

            logits = logreg(x)
            predictions = torch.argmax(logits, dim=1)
            
            total += y.size(0)
            correct += (predictions == y).sum().item()
            
        acc = 100 * correct / total
        print(f"Testing accuracy: {np.mean(acc)}")

  0%|          | 0/100 [00:00<?, ?it/s]

Testing accuracy: 9.716
Testing accuracy: 16.582
Testing accuracy: 19.074
Testing accuracy: 19.956
Testing accuracy: 20.944
Testing accuracy: 21.068
Testing accuracy: 21.524
Testing accuracy: 21.946
Testing accuracy: 21.916
Testing accuracy: 22.104


In [23]:
total = 0

correct = 0
for x, y in test_loader:
    x = x.to(device)
    y = y.to(device)

    logits = logreg(x)
    predictions = torch.argmax(logits, dim=1)

    total += y.size(0)
    correct += (predictions == y).sum().item()

acc = 100 * correct / total
print(f"Testing accuracy: {np.mean(acc)}")

Testing accuracy: 22.062
